CSC 575 Final Project - Vicky Lee / Implementation of Collaborative Filtering & Content-Based Recommender Systems Using Yelp Dataset

Preprocessing of Yelp Dataset to Reviews for Restaurants in Toronto with User, Item, Rating Columns

In [1]:
# Preprocessing of Yelp Dataset

# Set directory to Yelp dataset folder
import os
os.chdir("Documents/yelp/")

# Import libraries
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
from __future__ import division
import time

# Read review dataset
review = pd.read_csv("yelp_academic_dataset_review.csv")
# review.shape

# Count number of unqiue users and items for review dataset
n_users_review = review.user_id.unique().shape[0]
n_items_review = review.business_id.unique().shape[0]
# n_users_review
# n_items_review

# Read business dataset
business = pd.read_csv("yelp_academic_dataset_business.csv")
# business.shape

# Group businesses by 'city'
city = business.groupby('city')['city'].count()
# city.sort_values(ascending=False)

# Subset business to category 'Restaurants'
restaurant = business[business['categories'].str.contains("Restaurants",na=False)]
# restaurant.shape
# business['categories'].isnull().values.ravel().sum()

# Group restaurant businesses by city
city2 = restaurant.groupby('city')['city'].count()
# city2.sort_values(ascending=False)

# Filter restaurant businesses to city 'Toronto'
restaurant_toronto = restaurant.loc[restaurant['city'] == 'Toronto']
# restaurant_toronto.shape

# Left join restaruants in toronto table with review table
review_rest_tor = pd.merge(restaurant_toronto, review, on='business_id', how='left')
# review_rest_tor.shape
# review_rest_tor.columns

# Subset to user, item, rating columns
uir = review_rest_tor[['user_id','business_id','stars_y']]
# len(uir)

# Assign index for user and item
user_index = uir.user_id.unique()
item_index = uir.business_id.unique()

# Count number of unique users and items
n_users = uir.user_id.unique().shape[0]
n_items = uir.business_id.unique().shape[0]

Create User, Item Matrix and Filter down the dataset to 5 Core with every user and item with at least 5 ratings

In [2]:
# Split User, Item, Rating dataset to train and test sets of 70% & 30%
from sklearn.model_selection import train_test_split
train, test = train_test_split(uir, test_size=0.30, random_state=42)
# len(train)
# len(test)

# Create table for train data with list of users as index & items as columns
train_matrix = pd.DataFrame(index=user_index, columns=item_index)
# train_matrix.shape

# Fill in train_matrix table with ratings
for row in train.itertuples():
    user = row[1]
    item = row[2]
    train_matrix.loc[user][item] = row[3]  

# Create table for test data with list of users as index & items as columns    
test_matrix = pd.DataFrame(index=user_index, columns=item_index)
# test_matrix.shape

# Fill in test_matrix table with ratings
for row in test.itertuples():
    user = row[1]
    item = row[2]
    test_matrix.loc[user][item] = row[3]

# Begin filtering process to create 5 Core Subset

# Count number of rated items for each user
item_1 = train_matrix.apply(lambda x: x > 0, raw=True).sum(axis=1)
# item_1.value_counts()

# Filter down to the users with greater than or equal to 5 ratings
train1 = train_matrix
train1['item_1'] = item_1
train2 = train1.loc[train1['item_1'] >= 5]
# train2.shape

# Count number of rated users for each item
train2 = train2.drop('item_1',axis=1)
train3 = train2.transpose()
user_1 = train3.apply(lambda x: x > 0, raw=True).sum(axis=1)
# user_1.value_counts()

# Filter down to the items with greater than or equal to 5 ratings
train3['user_1'] = user_1
train4 = train3.loc[train3['user_1'] >= 5]
train4 = train4.drop('user_1',axis=1)
train5 = train4.transpose()
# train5.shape

# Repeat the process for both user and item
item_2 = train5.apply(lambda x: x > 0, raw=True).sum(axis=1)
# item_2.value_counts()
# item_2.shape
train5['item_2'] = item_2
train6 = train5.loc[train5['item_2'] >= 5]
train6 = train6.drop('item_2',axis=1)
train7 = train6.transpose()
user_2 = train7.apply(lambda x: x > 0, raw=True).sum(axis=1)
# user_2.value_counts()
train7['user_2'] = user_2
train8 = train7.loc[train7['user_2'] >= 5]
train8 = train8.drop('user_2',axis=1)
train9 = train8.transpose()
# train9.shape

# Check every user and item has at least 5 ratings
item_3 = train9.apply(lambda x: x > 0, raw=True).sum(axis=1)
# item_3.value_counts()
user_3 = train9.apply(lambda x: x > 0, raw=True).sum(axis=1)
# user_3.value_counts()

# Filter down the test matrix to filtered user and item in train matrix
test9 = test_matrix.loc[train9.index,train9.columns]
# test9.shape

Repeat the same process for Restaurants in Phoenix

In [3]:
# Filter restaurant businesses to city 'Phoenix'
restaurant_phoenix = restaurant.loc[restaurant['city'] == 'Phoenix']

# restaurant_phoenix.shape

# Left join restaruants in phoenix table with review table
review_rest_pho = pd.merge(restaurant_phoenix, review, on='business_id', how='left')
# review_rest_pho.shape
# review_rest_pho.columns

# Subset to user, item, rating columns
uir_ph = review_rest_pho[['user_id','business_id','stars_y']]
# len(uir_ph)

# Assign index for user and item
user_index_ph = uir_ph.user_id.unique()
item_index_ph = uir_ph.business_id.unique()

# Count number of unique users and items
n_users_ph = uir_ph.user_id.unique().shape[0]
n_items_ph = uir_ph.business_id.unique().shape[0]

# Split User, Item, Rating dataset to train and test sets of 70% & 30%
from sklearn.model_selection import train_test_split
train_ph, test_ph = train_test_split(uir_ph, test_size=0.30, random_state=42)
# len(train_ph)
# len(test_ph)

# Create table for train data with list of users as index & items as columns
train_matrix_ph = pd.DataFrame(index=user_index_ph, columns=item_index_ph)
# train_matrix_ph.shape

# Fill in train_matrix table with ratings
for row in train_ph.itertuples():
    user = row[1]
    item = row[2]
    train_matrix_ph.loc[user][item] = row[3]  

# Create table for test data with list of users as index & items as columns    
test_matrix_ph = pd.DataFrame(index=user_index_ph, columns=item_index_ph)
# test_matrix_ph.shape

# Fill in test_matrix table with ratings
for row in test_ph.itertuples():
    user = row[1]
    item = row[2]
    test_matrix_ph.loc[user][item] = row[3]

# Begin filtering process to create 5 Core Subset

# Count number of rated items for each user
item_1_ph = train_matrix_ph.apply(lambda x: x > 0, raw=True).sum(axis=1)
# item_1_ph.value_counts()

# Filter down to the users with greater than or equal to 5 ratings
train1_ph = train_matrix_ph
train1_ph['item_1'] = item_1_ph
train2_ph = train1_ph.loc[train1_ph['item_1'] >= 5]
# train2_ph.shape

# Count number of rated users for each item
train2_ph = train2_ph.drop('item_1',axis=1)
train3_ph = train2_ph.transpose()
user_1_ph = train3_ph.apply(lambda x: x > 0, raw=True).sum(axis=1)
# user_1_ph.value_counts()

# Filter down to the items with greater than or equal to 5 ratings
train3_ph['user_1'] = user_1_ph
train4_ph = train3_ph.loc[train3_ph['user_1'] >= 5]
train4_ph = train4_ph.drop('user_1',axis=1)
train5_ph = train4_ph.transpose()
# train5_ph.shape

# Repeat the process for both user and item
item_2_ph = train5_ph.apply(lambda x: x > 0, raw=True).sum(axis=1)
# item_2_ph.value_counts()

train5_ph['item_2'] = item_2_ph
train6_ph = train5_ph.loc[train5_ph['item_2'] >= 5]
train6_ph = train6_ph.drop('item_2',axis=1)
train7_ph = train6_ph.transpose()
user_2_ph = train7_ph.apply(lambda x: x > 0, raw=True).sum(axis=1)
# user_2_ph.value_counts()
train7_ph['user_2'] = user_2_ph
train8_ph = train7_ph.loc[train7_ph['user_2'] >= 5]
train8_ph = train8_ph.drop('user_2',axis=1)
train9_ph = train8_ph.transpose()
# train9_ph.shape

item_3_ph = train9_ph.apply(lambda x: x > 0, raw=True).sum(axis=1)
# item_3_ph.value_counts()

user_3_ph = train9_ph.apply(lambda x: x > 0, raw=True).sum(axis=1)
# user_3_ph.value_counts()

# Filter down the test matrix to filtered user and item in train matrix
test9_ph = test_matrix_ph.loc[train9_ph.index,train9_ph.columns]
# test9_ph.shape

Original Yelp dataset & User, Item, Rating dataset Summary

In [4]:
print '\nYelp Review Dataset'
print 'Size of original Yelp dataset, Review is %s ' % str(review.shape)
print 'Number of users is %s, number of items is %s ' % (str(n_users_review),str(n_items_review))
print '\nYelp Business Dataset'
print 'Size of original Yelp dataset, Business is %s ' % str(business.shape)
print 'Size of Restaurants subset of Business dataset is %s ' % str(restaurant.shape)
print '\nTop 10 Cities with the most number of restaurants are as follows \n%s ' % str(city2.sort_values(ascending=False).head(10))
print '\nRestaurants in Toronto Subset'
print 'Size of Restaurants in Toronto subset of Business dataset is %s ' % str(restaurant_toronto.shape)
print 'User, Item, Rating dataset for restuarnats in Toronto contain %s rows ' % str(len(uir))
print 'Number of users is %s, number of items is %s ' % (str(n_users),str(n_items))
print '\nRestaurants in Phoenix Subset'
print 'Size of Restaurants in Phoenix subset of Business dataset is %s ' % str(restaurant_phoenix.shape)
print 'User, Item, Rating dataset for restuarnats in Toronto contain %s rows ' % str(len(uir_ph))
print 'Number of users is %s, number of items is %s ' % (str(n_users_ph),str(n_items_ph))


Yelp Review Dataset
Size of original Yelp dataset, Review is (4153150, 10) 
Number of users is 1029432, number of items is 144072 

Yelp Business Dataset
Size of original Yelp dataset, Business is (144072, 16) 
Size of Restaurants subset of Business dataset is (48485, 16) 

Top 10 Cities with the most number of restaurants are as follows 
city
Toronto        6347
Las Vegas      5431
Phoenix        3353
Montréal       2852
Charlotte      2201
Pittsburgh     1990
Edinburgh      1412
Scottsdale     1356
Cleveland      1235
Mississauga    1128
Name: city, dtype: int64 

Restaurants in Toronto Subset
Size of Restaurants in Toronto subset of Business dataset is (6347, 16) 
User, Item, Rating dataset for restuarnats in Toronto contain 245127 rows 
Number of users is 58355, number of items is 6347 

Restaurants in Phoenix Subset
Size of Restaurants in Phoenix subset of Business dataset is (3353, 16) 
User, Item, Rating dataset for restuarnats in Toronto contain 266766 rows 
Number of users is

Train, Test Matrices of Users, Items & 5 Core Subset Summary

In [5]:
print '\nRestaurants in Toronto'
print '\nTrain dataset contains %s rows, Test dataset contains %s rows ' % (str(len(train)),str(len(test)))
print 'Size of train & test matrices with users in rows & items in columns is %s ' % (str(train_matrix.shape))
print '\nUsers of filtered 5 core subset has rated at least 5 items as follows  \n%s' % (str(item_3.value_counts().head(5)))
print '\nItems of filtered 5 core subset has been rated by at least 5 users as follows  \n%s' % (str(user_3.value_counts().head(5)))
print '\nSize of 5 core subset is %s ' % (str(train9.shape))
print '\nRestaurants in Phoenix'
print '\nTrain dataset contains %s rows, Test dataset contains %s rows ' % (str(len(train_ph)),str(len(test_ph)))
print 'Size of train & test matrices with users in rows & items in columns is %s ' % (str(train_matrix_ph.shape))
print '\nSize of 5 core subset is %s ' % (str(train9_ph.shape))


Restaurants in Toronto

Train dataset contains 171588 rows, Test dataset contains 73539 rows 
Size of train & test matrices with users in rows & items in columns is (58355, 6348) 

Users of filtered 5 core subset has rated at least 5 items as follows  
5    1419
6     946
7     663
8     568
9     436
dtype: int64

Items of filtered 5 core subset has been rated by at least 5 users as follows  
5    1419
6     946
7     663
8     568
9     436
dtype: int64

Size of 5 core subset is (7016, 3948) 

Restaurants in Phoenix

Train dataset contains 186736 rows, Test dataset contains 80030 rows 
Size of train & test matrices with users in rows & items in columns is (97476, 3354) 

Size of 5 core subset is (7032, 2153) 


Neighborhood-Based Collaborative Filtering - User Based & Item Based 

In [6]:
def collaborative_filtering(train, test, sim='cosine', type='user', knn=5):

    # Fill NAN values in train & test data as 0's
    train_0 = train.fillna(0)
    test_0 = test.fillna(0)

    # Create a similarity matrix of either users or items based on cosine or 
    # pearson correlation measure
    if sim == 'cosine':
        user_dist = pairwise_distances(train_0, metric='cosine')
        item_dist = pairwise_distances(train_0.T, metric='cosine')

    elif sim == 'pearson':
        user_dist = pairwise_distances(train_0, metric='correlation')
        item_dist = pairwise_distances(train_0.T, metric='correlation')
        
    user_sim = 1 - user_dist
    item_sim = 1 - item_dist
 
    # Create a dataframe with mean user ratings
    mean_rating = train.mean(axis=1)
    mean_user_rating = pd.concat([mean_rating] * len(train.columns), axis=1)
    mean_user_rating.columns = train.columns
    
    # Modify a dataframe so that mean user ratings are present only in matrix 
    # positions of rated items and 0's in matrix positions of non-rated items
    mean_user_rating_0 = mean_user_rating
    mean_user_rating_0[train_0 == 0] = 0
    
    # Normalize every user's ratings to mean of zero
    ratings_diff = train_0 - mean_user_rating_0
    
    # Create a dataframe with user's mean user ratings present in all items
    mean_user_rating_f = pd.concat([mean_rating] * len(train.columns), axis=1)
    mean_user_rating_f.columns = train.columns
    
    # Create a placeholder dataframe for predictions of rated items in test data
    pred = pd.DataFrame(index=train.index, columns=train.columns)
    
    # User-Based Collaborative Filtering
    if type == 'user':    
        # Index user similarity matrix with user ids for both rows and columns
        user_sim = pd.DataFrame(user_sim, index=train.index, columns=train.index)
        # When the number of k neareast neighbors is specified
        if knn != 'all':
            user_sim_mat = user_sim.as_matrix()
        
            # Item id's of rated items in test data for each user
            cols = test.columns
            test_rated = test.apply(lambda x: x > 0, raw=True).apply(lambda x: list(cols[x.values]), axis=1)
            
            # Iterate over each user, m
            for m in range(len(user_sim)):
                # Retrieve column of user m's similarities to all other users
                temp = user_sim_mat[m]
                temp = pd.DataFrame(temp,index=train.index, columns=['similarity'])
                # Rank user m's similarities
                temp['rank'] = temp['similarity'].rank(ascending=0)
                
                # Iterate over user m's rated items
                for n in range(len(test_rated[m])):
                    # For user m's nth rated item, extract column of ratings of 
                    # all users corresponding to nth item
                    temp2 = ratings_diff[[test_rated[m][n]]]
                    temp2.columns = ['rating']
                    # Contatenate similarity, rank, rating as one dataframe
                    result = pd.concat([temp, temp2], axis=1)    
                    # Filter down to the users who rated the items
                    result2 = result[result['rating'] != 0]
                    # Filter down to knn number of users with the knn 
                    # highest similairites
                    result3 = result2.nsmallest(int(knn), 'rank')
                
                    # Divide weighted sum of user's knn nearest neighbors' ratings by 
                    # sum of their similarities
                    score = result3['similarity'].dot(result3['rating'])/result3['similarity'].sum()
                    mean = mean_user_rating_f.loc[train.index[m]][test_rated[m][n]]
                
                    # Make a prediction by adding user's mean rating to weighted sum
                    pred.loc[train.index[m]][test_rated[m][n]] = mean + score
                    
        # When the number of k neareast neighbors is not specified and all 
        # available neighbors are used for prediction     
        elif knn == 'all':  
            # Compute user similarity weighted sum of available ratings of 
            # user's every neighbor 
            num_user = user_sim.dot(ratings_diff)
            
            # Sum user similarities    
            sum_sim_user = user_sim.sum(axis=1)
            sum_sim_mat_user = pd.concat([sum_sim_user] * len(train.columns), axis=1)
            sum_sim_mat_user.columns = train.columns

            # Create a dataframe of predictions computed by adding mean user 
            # rating to user similairty weighted sum of user's ratings 
            # divided by sum of user similarities
            pred = mean_user_rating_f + num_user / sum_sim_mat_user
                
    # Item-Based Collaborative Filtering  
    elif type == 'item':
        # Index item similarity matrix with item ids for both rows and columns
        item_sim = pd.DataFrame(item_sim, index=train.columns, columns=train.columns)
        
        # When the number of k neareast neighbors is specified
        if knn != 'all':
            item_sim_mat = item_sim.as_matrix()

            # User id's of rated items in test data for each item
            cols2 = test.T.columns
            test_rated2 = test.T.apply(lambda x: x > 0, raw=True).apply(lambda x: list(cols2[x.values]), axis=1)
        
            # Iterate over each item, m
            for m in range(len(item_sim)):
                # Retrieve column of item m's similarities to all other items
                temp = item_sim_mat[m]
                temp = pd.DataFrame(temp,index=train.columns, columns=['similarity'])
                # Rank item m's similarities
                temp['rank'] = temp['similarity'].rank(ascending=0)

                # Iterate over item m's rated users
                for n in range(len(test_rated2[m])):
                    # For item m's nth rated user, extract column of ratings of 
                    # all items corresponding to nth user
                    temp2 = ratings_diff.T[[test_rated2[m][n]]]
                    temp2.columns = ['rating']
                    # Contatenate similarity, rank, rating as one dataframe
                    result = pd.concat([temp, temp2], axis=1)    
                    # Filter down to the items that are rated by the user
                    result2 = result[result['rating'] != 0]
                    # Filter down to knn number of items with the knn 
                    # highest similairites
                    result3 = result2.nsmallest(int(knn), 'rank')
                
                    # Divide weighted sum of item's knn nearest neighbors' 
                    # ratings by sum of their similarities
                    score = result3['similarity'].dot(result3['rating'])/result3['similarity'].sum()
                    mean = mean_user_rating_f.loc[test_rated2[m][n]][train.columns[m]]
                
                    # Make a prediction by adding user's mean rating to weighted sum
                    pred.loc[test_rated2[m][n]][train.columns[m]] = mean + score
        
        # When the number of k neareast neighbors is not specified and 
        # all available neighbors are used for prediction
        elif knn == 'all':
            # Compute item similarity weighted sum of available ratings of 
            # item's every neighbor
            num_item = ratings_diff.dot(item_sim)

            # Sum item similarities 
            sum_sim_item = item_sim.sum(axis=1)
            sum_sim_item = pd.DataFrame(sum_sim_item, index=train.columns)
            sum_sim_mat_item = pd.concat([sum_sim_item.T] * len(train), axis=0)

            sum_sim_mat_item.index = train.index

            # Create a dataframe of predictions computed by adding mean user 
            # rating to item similairty weighted sum of items' ratings 
            # divided by sum of item similarities
            pred = mean_user_rating_f + num_item / sum_sim_mat_item
            
    pred_0 = pred.fillna(0)
    pred_0_mat = pred_0.as_matrix()            
    return pred_0_mat

In [7]:
test9_0 = test9.fillna(0)
test9_0_mat = test9_0.as_matrix()
test9_0_ph = test9_ph.fillna(0)
test9_0_mat_ph = test9_0_ph.as_matrix()

Mean Absolute Error Computation Function

In [8]:
from sklearn.metrics import mean_absolute_error
def mae(prediction, actual):
    prediction = prediction[actual.nonzero()].flatten() 
    actual = actual[actual.nonzero()].flatten()
    return mean_absolute_error(prediction, actual)

Run Collaborative Filtering on Restaurants in Toronto

In [9]:
start_time = time.time()
pred_user_5 = collaborative_filtering(train9, test9, sim='cosine', type='user', knn='5')
pred_user_10 = collaborative_filtering(train9, test9, sim='cosine', type='user', knn='10')
pred_user_20 = collaborative_filtering(train9, test9, sim='cosine', type='user', knn='20')
pred_user_all = collaborative_filtering(train9, test9, sim='cosine', type='user', knn='all')
pred_item_5 = collaborative_filtering(train9, test9, sim='cosine', type='item', knn='5')
pred_item_10 = collaborative_filtering(train9, test9, sim='cosine', type='item', knn='10')
pred_item_20 = collaborative_filtering(train9, test9, sim='cosine', type='item', knn='20')
pred_item_all = collaborative_filtering(train9, test9, sim='cosine', type='item', knn='all')
print '8 Runs of Collaborative Filtering for Restaurant in Toronto took %s seconds' % (time.time() - start_time)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:76: RuntimeWarning: invalid value encountered in double_scalars
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:136: RuntimeWarning: invalid value encountered in double_scalars


8 Runs of Collaborative Filtering for Restaurant in Toronto took 2022.19950795 seconds


In [10]:
start_time = time.time()
pred_user_cosine = collaborative_filtering(train9, test9, sim='cosine', type='user', knn='all')
pred_user_pearson = collaborative_filtering(train9, test9, sim='pearson', type='user', knn='all')
pred_item_cosine = collaborative_filtering(train9, test9, sim='cosine', type='item', knn='all')
pred_item_pearson = collaborative_filtering(train9, test9, sim='pearson', type='item', knn='all')
print '4 Runs of Collaborative Filtering for Restaurant in Toronto took %s seconds' % (time.time() - start_time)

4 Runs of Collaborative Filtering for Restaurant in Toronto took 552.532515049 seconds


Run Collaborative Filtering on Restaurants in Phoenix

In [11]:
start_time = time.time()
pred_user_5_ph = collaborative_filtering(train9_ph, test9_ph, sim='cosine', type='user', knn='5')
pred_user_10_ph = collaborative_filtering(train9_ph, test9_ph, sim='cosine', type='user', knn='10')
pred_user_20_ph = collaborative_filtering(train9_ph, test9_ph, sim='cosine', type='user', knn='20')
pred_user_all_ph = collaborative_filtering(train9_ph, test9_ph, sim='cosine', type='user', knn='all')
pred_item_5_ph = collaborative_filtering(train9_ph, test9_ph, sim='cosine', type='item', knn='5')
pred_item_10_ph = collaborative_filtering(train9_ph, test9_ph, sim='cosine', type='item', knn='10')
pred_item_20_ph = collaborative_filtering(train9_ph, test9_ph, sim='cosine', type='item', knn='20')
pred_item_all_ph = collaborative_filtering(train9_ph, test9_ph, sim='cosine', type='item', knn='all')
print '8 Runs of Collaborative Filtering for Restaurant in Phoenix took %s seconds' % (time.time() - start_time)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:76: RuntimeWarning: invalid value encountered in double_scalars
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:136: RuntimeWarning: invalid value encountered in double_scalars


8 Runs of Collaborative Filtering for Restaurant in Phoenix took 1469.81145906 seconds


In [12]:
start_time = time.time()
pred_user_cosine_ph = collaborative_filtering(train9_ph, test9_ph, sim='cosine', type='user', knn='all')
pred_user_pearson_ph = collaborative_filtering(train9_ph, test9_ph, sim='pearson', type='user', knn='all')
pred_item_cosine_ph = collaborative_filtering(train9_ph, test9_ph, sim='cosine', type='item', knn='all')
pred_item_pearson_ph = collaborative_filtering(train9_ph, test9_ph, sim='pearson', type='item', knn='all')
print '4 Runs of Collaborative Filtering for Restaurant in Phoenix took %s seconds' % (time.time() - start_time)

4 Runs of Collaborative Filtering for Restaurant in Phoenix took 259.879482031 seconds


Results for Restaurants in Toronto

In [13]:
print 'Restaurnats in Toronto with Varying Number of Nearest Neighbors'
print 'User-based CF using 5 nearest neighbors MAE: ' + str(round(mae(pred_user_5, test9_0_mat),4))
print 'User-based CF using 10 nearest neighbors MAE: ' + str(round(mae(pred_user_10, test9_0_mat),4))
print 'User-based CF using 20 nearest neighbors MAE: ' + str(round(mae(pred_user_20, test9_0_mat),4))
print 'User-based CF using All available neighbors MAE: ' + str(round(mae(pred_user_all, test9_0_mat),4))
print 'Item-based CF using 5 nearest neighbors MAE: ' + str(round(mae(pred_item_5, test9_0_mat),4))
print 'Item-based CF using 10 nearest neighbors MAE: ' + str(round(mae(pred_item_10, test9_0_mat),4))
print 'Item-based CF using 20 nearest neighbors MAE: ' + str(round(mae(pred_item_20, test9_0_mat),4))
print 'Item-based CF using All available neighbors MAE: ' + str(round(mae(pred_item_all, test9_0_mat),4))

Restaurnats in Toronto with Varying Number of Nearest Neighbors
User-based CF using 5 nearest neighbors MAE: 0.9265
User-based CF using 10 nearest neighbors MAE: 0.9044
User-based CF using 20 nearest neighbors MAE: 0.8971
User-based CF using All available neighbors MAE: 0.8717
Item-based CF using 5 nearest neighbors MAE: 0.988
Item-based CF using 10 nearest neighbors MAE: 0.9705
Item-based CF using 20 nearest neighbors MAE: 0.9666
Item-based CF using All available neighbors MAE: 0.8734


In [14]:
print 'Restaurnats in Toronto Using Cosine Similarity or Pearson Correlation'
print 'User-based CF using Cosine Similarity MAE: ' + str(round(mae(pred_user_cosine, test9_0_mat),4))
print 'User-based CF using Pearson Correlation MAE: ' + str(round(mae(pred_user_pearson, test9_0_mat),4))
print 'Item-based CF using Cosine Similarity MAE: ' + str(round(mae(pred_item_cosine, test9_0_mat),4))
print 'Item-based CF using Pearson Correlation MAE: ' + str(round(mae(pred_item_pearson, test9_0_mat),4))

Restaurnats in Toronto Using Cosine Similarity or Pearson Correlation
User-based CF using Cosine Similarity MAE: 0.8717
User-based CF using Pearson Correlation MAE: 0.8724
Item-based CF using Cosine Similarity MAE: 0.8734
Item-based CF using Pearson Correlation MAE: 0.8747


Results for Restaurant in Phoenix

In [15]:
print 'Restaurnats in Phoenix with Varying Number of Nearest Neighbors'
print 'User-based CF using 5 nearest neigbors MAE: ' + str(round(mae(pred_user_5_ph, test9_0_mat_ph),4))
print 'User-based CF using 10 nearest neigbors MAE: ' + str(round(mae(pred_user_10_ph, test9_0_mat_ph),4))
print 'User-based CF using 20 nearest neigbors MAE: ' + str(round(mae(pred_user_20_ph, test9_0_mat_ph),4))
print 'User-based CF using All available neigbors MAE: ' + str(round(mae(pred_user_all_ph, test9_0_mat_ph),4))
print 'Item-based CF using 5 nearest neigbors MAE: ' + str(round(mae(pred_item_5_ph, test9_0_mat_ph),4))
print 'Item-based CF using 10 nearest neigbors MAE: ' + str(round(mae(pred_item_10_ph, test9_0_mat_ph),4))
print 'Item-based CF using 20 nearest neigbors MAE: ' + str(round(mae(pred_item_20_ph, test9_0_mat_ph),4))
print 'Item-based CF using All available neigbors MAE: ' + str(round(mae(pred_item_all_ph, test9_0_mat_ph),4))

Restaurnats in Phoenix with Varying Number of Nearest Neighbors
User-based CF using 5 nearest neigbors MAE: 0.9366
User-based CF using 10 nearest neigbors MAE: 0.9091
User-based CF using 20 nearest neigbors MAE: 0.8979
User-based CF using All available neigbors MAE: 0.8832
Item-based CF using 5 nearest neigbors MAE: 0.9908
Item-based CF using 10 nearest neigbors MAE: 0.9747
Item-based CF using 20 nearest neigbors MAE: 0.9699
Item-based CF using All available neigbors MAE: 0.8846


In [16]:
print 'Restaurnats in Phoenix Using Cosine Similarity or Pearson Correlation'
print 'User-based CF using Cosine Similarity MAE: ' + str(round(mae(pred_user_cosine_ph, test9_0_mat_ph),4))
print 'User-based CF using Pearson Correlation MAE: ' + str(round(mae(pred_user_pearson_ph, test9_0_mat_ph),4))
print 'Item-based CF using Cosine Similarity MAE: ' + str(round(mae(pred_item_cosine_ph, test9_0_mat_ph),4))
print 'Item-based CF using Pearson Correlation MAE: ' + str(round(mae(pred_item_pearson_ph, test9_0_mat_ph),4))

Restaurnats in Phoenix Using Cosine Similarity or Pearson Correlation
User-based CF using Cosine Similarity MAE: 0.8832
User-based CF using Pearson Correlation MAE: 0.8951
Item-based CF using Cosine Similarity MAE: 0.8846
Item-based CF using Pearson Correlation MAE: 0.8859


Content-Based Recommender Systems

Preprocess features dataset for Restaurants in Toronto

In [17]:
feature = restaurant_toronto[['business_id','attributes','categories']]

# Extract features from column, categories & attributes for items(restaurants in Toronto)
feature1 = pd.DataFrame(feature['categories'].astype(str).str.split(',').tolist())
feature1 = feature1.apply(lambda x: x.str.lower())
feature1 = feature1.apply(lambda x: x.str.replace('u\'',''))
feature1 = feature1.apply(lambda x: x.str.replace('u\"',''))
feature1 = feature1.apply(lambda x: x.str.replace('\'',''))
feature1 = feature1.apply(lambda x: x.str.replace('[',''))
feature1 = feature1.apply(lambda x: x.str.replace(']',''))

# Extract features from column, attributes for items(restaurants in Toronto)
feature2 = pd.DataFrame(feature['attributes'].astype(str).str.split(',').tolist())
feature2 = feature2.apply(lambda x: x.str.lower())
feature2 = feature2.apply(lambda x: x.str.replace('u\'',''))
feature2 = feature2.apply(lambda x: x.str.replace('u\"',''))
feature2 = feature2.apply(lambda x: x.str.replace(r'.*: {',''))
feature2 = feature2.apply(lambda x: x.str.replace('}\"',''))
feature2 = feature2.apply(lambda x: x.str.replace('\'',''))
feature2 = feature2.apply(lambda x: x.str.replace(':',''))
feature2 = feature2.apply(lambda x: x.str.replace('[',''))
feature2 = feature2.apply(lambda x: x.str.replace(']',''))

feature1 = feature1.as_matrix()
feature2 = feature2.as_matrix()
features = np.concatenate((feature1, feature2),axis=1)
features = pd.DataFrame(features)

features.index = feature['business_id']
filtered = list(train9.columns.values)
features_filt = features[features.index.isin(filtered)]
number_of_features = pd.unique(features_filt.values.ravel())
# len(number_of_features)

Preprocess features dataset for Restaurants in Phoenix

In [18]:
feature_ph = restaurant_phoenix[['business_id','attributes','categories']]

# Extract features from column, categories & attributes for items(restaurants in Phoenix)
feature1_ph = pd.DataFrame(feature_ph['categories'].astype(str).str.split(',').tolist())
feature1_ph = feature1_ph.apply(lambda x: x.str.lower())
feature1_ph = feature1_ph.apply(lambda x: x.str.replace('u\'',''))
feature1_ph = feature1_ph.apply(lambda x: x.str.replace('u\"',''))
feature1_ph = feature1_ph.apply(lambda x: x.str.replace('\'',''))
feature1_ph = feature1_ph.apply(lambda x: x.str.replace('[',''))
feature1_ph = feature1_ph.apply(lambda x: x.str.replace(']',''))

# Extract features from column, attributes for items(restaurants in Phoenix)
feature2_ph = pd.DataFrame(feature_ph['attributes'].astype(str).str.split(',').tolist())
feature2_ph = feature2_ph.apply(lambda x: x.str.lower())
feature2_ph = feature2_ph.apply(lambda x: x.str.replace('u\'',''))
feature2_ph = feature2_ph.apply(lambda x: x.str.replace('u\"',''))
feature2_ph = feature2_ph.apply(lambda x: x.str.replace(r'.*: {',''))
feature2_ph = feature2_ph.apply(lambda x: x.str.replace('}\"',''))
feature2_ph = feature2_ph.apply(lambda x: x.str.replace('\'',''))
feature2_ph = feature2_ph.apply(lambda x: x.str.replace(':',''))
feature2_ph = feature2_ph.apply(lambda x: x.str.replace('[',''))
feature2_ph = feature2_ph.apply(lambda x: x.str.replace(']',''))

feature1_ph = feature1_ph.as_matrix()
feature2_ph = feature2_ph.as_matrix()
features_ph = np.concatenate((feature1_ph, feature2_ph),axis=1)
features_ph = pd.DataFrame(features_ph)

features_ph.index = feature_ph['business_id']
filtered_ph = list(train9_ph.columns.values)
features_filt_ph = features_ph[features_ph.index.isin(filtered_ph)]
number_of_features_ph = pd.unique(features_filt_ph.values.ravel())
# len(number_of_features_ph)

Features Summary

In [19]:
print 'Restaurants in Toronto'
print '\nThere are %s features' % str(len(number_of_features))
print 'Examples of features are as follows \n%s ' % str(number_of_features[50:70])
print '\nRestaurants in Phoenix'
print '\nThere are %s features' % str(len(number_of_features_ph))
print 'Examples of features are as follows \n%s ' % str(number_of_features_ph[350:370])

Restaurants in Toronto

There are 536 features
Examples of features are as follows 
[' lunch false' ' dinner false' ' breakfast true' ' brunch true'
 ' hastv true' ' noiselevel average' ' restaurantspricerange2 1'
 ' restaurantsreservations true' ' restaurantstableservice true'
 ' wheelchairaccessible true' ' wifi free' 'restaurants' ' japanese'
 ' sushi bars' 'alcohol beer_and_wine' ' restaurantsdelivery true' 'korean'
 ' street true' ' caters false' ' goodforkids false'] 

Restaurants in Phoenix

There are 476 features
Examples of features are as follows 
[' kosher true' ' halal true' ' soy-free true' 'karaoke' 'latin american'
 ' cambodian' 'tapas/small plates' ' szechuan' ' local flavor'
 ' fruits & veggies' 'drugstores' 'bikeparking false' 'sports clubs'
 ' food stands' ' jazz & blues' 'caribbean' ' cuban' 'arts & entertainment'
 ' cupcakes' 'fondue'] 


Content-Based Naive Bayes Approach

In [20]:
def content_based_recommender(train, test, features):

    # Train Data
    # For each user, retrieve item id's of rated items
    col = train.columns
    pref_known_train = train.apply(lambda x: x > 0, raw=True).apply(lambda x: list(col[x.values]), axis=1)

    # Test Data
    # For each user, retrieve item id's of rated items
    cols = test.columns
    pref_known_test = test.apply(lambda x: x > 0, raw=True).apply(lambda x: list(cols[x.values]), axis=1)

    # Create a dataframe with mean user ratings
    mean_rating = train.mean(axis=1)
    mean_user_rating = pd.concat([mean_rating] * len(train.columns), axis=1)
    mean_user_rating.columns = train.columns
    
    # Create a dataframe for test data prediction
    test_pred = pd.DataFrame(index=test.index, columns=test.columns)

    # For each user m, create a dataframe with columns, 'feature', 'like', 'dislike'
    for m in range(len(train)):
        like_m = pd.DataFrame(data=None, columns=['feature','like','dislike'])
        like = 0
        dislike = 0
        total = 0
        # Iterate over items with ratings
        for x1 in pref_known_train[m]:
            # Count the item, x1 as 'like' if the rating is greater than 2
            if train.loc[train.index[m]][x1] > 2:
                like = like + 1
                # Iterate over features of item, x1 and append row with feature: z1, 
                # 1 for like, 0 for dislike
                for z1 in np.array(features.loc[x1]):
                    if z1 is not None:
                        like_new1 = pd.DataFrame([[z1,1,0]],columns=['feature','like','dislike'])
                        like_m = like_m.append(like_new1)
                    
            # Count the item, x1 as 'dislike' if the rating is less than 2
            else:
                dislike = dislike + 1
                # Iterate over features of item, x1 and append row with feature: z1, 
                # 0 for like, 1 for dislike
                for z1 in np.array(features.loc[x1]):
                    if z1 is not None:
                        like_new1 = pd.DataFrame([[z1,0,1]],columns=['feature','like','dislike'])
                        like_m = like_m.append(like_new1)
            total = total + 1
    
        # P(like) & P(dislike) for user m
        prob_like =  like / total
        prob_dislike = dislike / total
        # Sum like & dislike counts by features
        like_m_f = like_m.groupby("feature").sum()

        # Conditional probabilities, P(feature|Like) & P(feature|Dislike) 
        # for each feature with beta=0.1
        like_m_f['pL'] = (like_m_f['like'] + 0.1) / (like + 2*(0.1))
        like_m_f['pDL'] = (like_m_f['dislike'] + 0.1) / (dislike + 2*(0.1))

        # Iterate over items to be predicted
        for x2 in pref_known_test[m]:
            like_m_l = pd.DataFrame(data=None, columns=['feature'])
            # Iterate over features of item, x2 and add feature, z2 
            for z2 in np.array(features.loc[x2]):
                if z2 is not None:
                    like_new2 = pd.DataFrame([[z2]],columns=['feature'])
                    like_m_l = like_m_l.append(like_new2)
        
            # Filter down to unique feature list
            like_m_l_f = like_m_l.feature.unique()

            # Add features and its conditional probabilities that are newly compiled 
            # in test data as 0 for like & dislike
            for w in like_m_l_f:
                if w not in like_m_f.index:
                    like_new = pd.DataFrame([[0,0,0.1/(like+(2*0.1)),0.1/(dislike+(2*0.1))]],index=[w],columns=['like','dislike','pL','pDL'])
                    like_m_f = like_m_f.append(like_new) 
        
            # Compute P(Liked) * Product of P(feature|Like) for all features 
            pLiked = prob_like
            for v in np.array(features.loc[x2]):
                if v is not None:
                    pLiked = pLiked * like_m_f.loc[str(v)]['pL']
                else:
                    pass
            
            # Compute P(Disliked) * Product of P(feature|Dislike) for all features 
            pDisliked = prob_dislike
            for v2 in np.array(features.loc[x2]):
                if v2 is not None:
                    pDisliked = pDisliked * like_m_f.loc[str(v2)]['pDL']
                else:
                    pass
    
            # Compute P1 as P(Liked/features) & P2 as P(Disliked/features) 
            P1 = pLiked / (pLiked+pDisliked)
            P2 = pDisliked / (pLiked+pDisliked)
            val = np.log(P1/P2)
            # Normalize to 1-5 rating scale
            rat = 1/(1+np.exp(-val))
            rating = rat*4 +1 
            rating

            # Assign rating as prediction for item, x2 for user m
            test_pred.loc[train.index[m]][x2] = rating

    return test_pred

In [21]:
start_time = time.time()
test_pred = content_based_recommender(train9, test9, features_filt)
print 'Content Based Recommender for Restaurant in Toronto took %s seconds' % (time.time() - start_time)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:99: RuntimeWarning: divide by zero encountered in double_scalars
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:99: RuntimeWarning: divide by zero encountered in log


Content Based Recommender for Restaurant in Toronto took 8280.69850421 seconds


In [22]:
start_time = time.time()
test_pred_ph = content_based_recommender(train9_ph, test9_ph, features_filt_ph)
print 'Content Based Recommender for Restaurant in Phoenix took %s seconds' % (time.time() - start_time)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:99: RuntimeWarning: divide by zero encountered in double_scalars
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:99: RuntimeWarning: divide by zero encountered in log


Content Based Recommender for Restaurant in Phoenix took 7056.17153001 seconds


In [23]:
test_pred_0 = test_pred.fillna(0)
test_pred_0_mat = test_pred_0.as_matrix()
pred_list = list(test_pred.values.ravel())
actual_list = list(test9.values.ravel())
# len([x for x in pred_list if x>0])
# len([x for x in actual_list if x>0])
test_pred_0_ph = test_pred_ph.fillna(0)
test_pred_0_mat_ph = test_pred_0_ph.as_matrix()
pred_list_ph = list(test_pred_ph.values.ravel())
actual_list_ph = list(test9_ph.values.ravel())
# len([x for x in pred_list_ph if x>0])
# len([x for x in actual_list_ph if x>0])
print 'Content-Based Recommenders Sytems for Toronto Mean Absolute Error: ' + str(round(mae(test_pred_0_mat,test9_0_mat),4))
print 'Content-Based Recommenders Sytems for Phoenix Mean Absolute Error: ' + str(round(mae(test_pred_0_mat_ph,test9_0_mat_ph),4))

Content-Based Recommenders Sytems for Toronto Mean Absolute Error: 1.5151
Content-Based Recommenders Sytems for Phoenix Mean Absolute Error: 1.4289
